In [1]:
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup
import cloudscraper

In [2]:
url = "https://www.realtor.com/realestateandhomes-search/Tampa_FL/show-recently-sold"
scraper = cloudscraper.create_scraper()

response = scraper.get(url)
#response = requests.get(url)

if not response.status_code == 200:
    None
try:
    results_page = BeautifulSoup(response.content,'lxml') 
    print('Success')
except:
    None

Success


In [50]:
realtor_list=list()
search = 1

while search:
    results = results_page.find('section',{'class':'jsx-766663426 srp-content'})
    # listing_page = results.find('ul',{'data-testid':'property-list-container'})
    listing_page = results.find('ul',{'class':'jsx-343105667 property-list list-unstyle'})
    listing_results=listing_page.find_all('li',{'data-testid':'result-card'})

    navigation = results.find('div',{'role':'navigation'})
    tabindex = navigation.find('a',{'aria-label':'Go to next page'}).get('tabindex')

    if tabindex == 0:
        next_page_url = navigation.find('a',{'aria-label':'Go to next page'}).get('href')
        next_page_response = scraper.get(next_page_url)
        if not next_page_response.status_code == 200:
            None
        else:
            next_pg = BeautifulSoup(next_page_response.content,'lxml')  
    
    for li in listing_results:
        root_url = 'https://www.realtor.com'
        listing_url = root_url + li.find('a',{'rel':'noopener'}).get('href')
        try:
            new_response = scraper.get(listing_url)
            if not new_response.status_code == 200:
                None
            else:
                soup = BeautifulSoup(new_response.content,'lxml')
                property_address = soup.find('h1',{'itemprop':'address'}).find('span',{'itemprop':'streetAddress'}).get_text()[:-1]
                sale_price=soup.find('div',{'class':'ldp-header-price'}).find_all('span')[1].get_text().strip()
                prop_price_history=soup.find('div',{'id':'ldp-history-price'}).find('tbody')
                list_price=prop_price_history.find_all('tr')[1].find_all('td')[2].get_text()
                key_prop_data = soup.find('div',{'id':'ldp-property-meta'})
                bedrooms = key_prop_data.find('li',{'data-label':'property-meta-beds'}).find('span',{'class':'data-value'}).get_text()
                baths = key_prop_data.find('li',{'data-label':'property-meta-bath'}).find('span',{'class':'data-value'}).get_text()
                sqft = key_prop_data.find('li',{'data-label':'property-meta-sqft'}).find('span',{'class':'data-value'}).get_text()
                lot_size = key_prop_data.find('li',{'data-label':'property-meta-lotsize'}).find('span',{'class':'data-value'}).get_text()

                property_features = soup.find('div',{'id':'ldp-details'}).find('div',{'class':'listing-subsection listing-subsection-features'})
                #features = property_features.find_all('div':{'class':''})
                all_features=property_features.find_all('div',{'class':'col-lg-3 col-sm-6 col-xs-6 col-xxs-12 ldp-features-image-tag'})

                exterior_features = all_features[2].find('ul',{'class':'list-default'}).find_all('li')
                ext_list = list()
                for i in range(len(exterior_features)):
                    ext_list.append(exterior_features[i].get_text())

                interior_features = property_features.find_all('div',{'class':'row'})[2].find_all('li')
                int_list = list()
                for i in range(len(interior_features)):
                    int_list.append(interior_features[i].get_text())

                appliance_features = property_features.find_all('div',{'class':'row'})[3].find_all('li')
                appliances = list()
                for i in range(len(appliance_features)):
                    appliances.append(appliance_features[i].get_text())

                heat_cool_features = property_features.find_all('div',{'class':'row'})[4].find_all('li')
                heat_cool_list = list()
                for i in range(len(heat_cool_features)):
                    heat_cool_list.append(heat_cool_features[i].get_text())

                pool_spa_features = property_features.find_all('div',{'class':'row'})[5].find_all('li')
                pool_spa_list = list()
                for i in range(len(pool_spa_features)):
                    pool_spa_list.append(pool_spa_features[i].get_text())

        except:
            None
        realtor_list.append([property_address,sale_price,list_price,bedrooms,baths,sqft,lot_size,ext_list,int_list,appliances,
                            heat_cool_list,pool_spa_list])
    else:
        search = 0

In [ ]:
def get_first_page(url, search=1):
    response = scraper.get(url)
    if not response.status_code == 200:
    None
    try:
        soup = BeautifulSoup(response.content,'lxml') 
        print('Success')
    except:
        None
        
    results = soup.find('section',{'class':'jsx-766663426 srp-content'})
    listing_page = results.find('ul',{'class':'jsx-343105667 property-list list-unstyle'})
    listing_results=listing_page.find_all('li',{'data-testid':'result-card'})


In [51]:
len(realtor_list)

42

In [52]:
realtor_list

[['14104 Stonebrook Ct',
  '$580,000',
  '$585,000',
  '4',
  '2.5',
  '2,526',
  '0.23',
  ['Fenced',
   'Irrigation System',
   'Lighting',
   'Outdoor Grill',
   'Outdoor Shower',
   'Rain Gutters',
   'Sliding Doors',
   'Fencing: Wood',
   'Patio And Porch Features: Covered, Deck, Screened',
   'Road Surface Type: Asphalt'],
  ['Attic Ventilator',
   'Built in Features',
   'Ceiling Fans(s)',
   'Eating Space In Kitchen',
   'High Ceiling(s)',
   'Kitchen/Family Room Combo',
   'Open Floorplan',
   'Solid Surface Counters',
   'Solid Wood Cabinets',
   'Split Bedroom',
   'Vaulted Ceiling(s)',
   'Walk-In Closet(s)',
   'Furnished Description: Unfurnished',
   'Flooring: Bamboo, Ceramic Tile, Wood',
   'Window Features: Blinds'],
  ['Dishwasher',
   'Disposal',
   'Dryer',
   'Electric Water Heater',
   'Exhaust Fan',
   'Ice Maker',
   'Microwave',
   'Range',
   'Range Hood',
   'Refrigerator',
   'Solar Hot Water Owned',
   'Washer',
   'Laundry Features: Inside, Laundry Room']

In [53]:
df = pd.DataFrame(columns=['Property Address','Sale Price','Listing Price','# Bedrooms','# Bathrooms',
                           'House Size (sqft)','Lot Size','External Features','Internal Features','Appliances',
                          'Heating & Cooling','Pool & Spa'])
for prop in realtor_list:
    address = prop[0]
    sale_price=prop[1]
    list_price=prop[2]
    beds = prop[3]
    baths = prop[4]
    sqft = prop[5]
    lotsize = prop[6]
    ext_feat=prop[7]
    int_feat=prop[8]
    app=prop[9]
    heat_cool=prop[10]
    pool_spa=prop[11]
    
    df = df.append({'Property Address':address,'Sale Price':sale_price,'Listing Price':list_price,
                    '# Bedrooms':beds,'# Bathrooms':baths,'House Size (sqft)':sqft,'Lot Size':lotsize,
                   'External Features':ext_feat,'Internal Features':int_feat,'Appliances':app,
                   'Heating & Cooling':heat_cool,'Pool & Spa':pool_spa},ignore_index=True)

#df = df.sort_values(by='Rank', ascending=True)

In [54]:
df

,Property Address,Sale Price,Listing Price,# Bedrooms,# Bathrooms,House Size (sqft),Lot Size,External Features,Internal Features,Appliances,Heating & Cooling,Pool & Spa
0,14104 Stonebrook Ct,"$580,000","$585,000",4,2.5,"2,526",0.23,"[Fenced, Irrigation System, Lighting, Outdoor ...","[Attic Ventilator, Built in Features, Ceiling ...","[Dishwasher, Disposal, Dryer, Electric Water H...","[Cooling Features: Central Air, Fireplace Feat...","[Pool Features: Pool Features: Deck, Gunite, I..."
1,6211 S Elberon St,"$900,000","$899,900",4,3.5,"2,746","5,663","[Balcony, Dog Run, Irrigation System, Rain Gut...","[Ceiling Fans(s), Crown Molding, Eating Space ...","[Built-In Oven, Dishwasher, Disposal, Dryer, E...","[Cooling Features: Central Air, Zoned, Firepla...","[Attached Garage: Yes, Garage Spaces: 2, Garag..."
2,4303 S Clark Ave,"$560,000","$549,900",3,2,"1,506","6,970","[Fenced, Lighting, Patio And Porch Features: F...","[Eating Space In Kitchen, Kitchen/Family Room ...","[Dishwasher, Disposal, Electric Water Heater, ...","[Cooling Features: Central Air, Heating Featur...","[Pool Features: Pool Features: Fiberglass, In ..."
3,12321 Bayou Flats Ln,"$364,900","$364,900",3,2.5,"1,816","1,742","[Sidewalk, Sliding Doors, Other Structures: Ke...","[Ceiling Fans(s), Eating Space In Kitchen, Hig...","[Dishwasher, Disposal, Electric Water Heater, ...","[Cooling Features: Central Air, Heating Featur...",[Water View: Pond]
4,13307 N Oregon Ave,"$316,000","$300,000",3,2,"1,176",0.25,"[Fenced, French Doors, Lighting, Fencing: Viny...","[Ceiling Fans(s), Crown Molding, Living Room/D...","[Dishwasher, Dryer, Microwave, Range, Refriger...","[Cooling Features: Central Air, Heating Featur...","[Water Body Name: DORSET LAKE, Water Features:..."
5,10623 Liberty Bell Dr,"$364,500","$350,000",4,2,"1,746","8,276","[Irrigation System, Sidewalk, Patio And Porch ...","[Ceiling Fans(s), Open Floorplan, Split Bedroo...","[Dishwasher, Dryer, Microwave, Range, Refriger...","[Cooling Features: Central Air, Heating Featur...","[Attached Garage: Yes, Garage Spaces: 3, Garag..."
6,3915 Fontainebleau Dr,"$835,000","$900,000",3,2,"1,548","9,148","[Sliding Doors, Road Surface Type: Paved]","[Ceiling Fans(s), Open Floorplan, Solid Wood C...","[Cooktop, Dishwasher, Disposal, Microwave, Ref...","[Cooling Features: Central Air, Heating Featur...","[Water Features: Canal - Saltwater, Water View..."
7,3017 W Bay View Ave Unit D,"$455,000","$455,000",3,2.5,"2,186",871,"[Fenced, Outdoor Grill, Road Surface Type: Paved]","[Ceiling Fans(s), Kitchen/Family Room Combo, L...","[Built-In Oven, Cooktop, Dishwasher, Disposal,...","[Cooling Features: Central Air, Heating Featur...","[Lot Size Acres: 0.02, Lot Size Square Feet: 871]"
8,6803 Santa Monica Ct,"$295,000","$285,000",3,2,"1,687",0.23,"[Fenced, French Doors, Fencing: Wood, Patio An...","[Ceiling Fans(s), Solid Surface Counters, Soli...","[Dishwasher, Microwave, Range, Refrigerator, L...","[Cooling Features: Central Air, Heating Featur...","[Lot Description: Cul-De-Sac, Lot Size Acres: ..."
9,18913 Duquesne Dr,"$262,000","$255,000",2,2.5,"1,759","1,307","[Sidewalk, Sliding Doors, Patio And Porch Feat...","[Eating Space In Kitchen, Living Room/Dining R...","[Dishwasher, Disposal, Electric Water Heater, ...","[Cooling Features: Central Air, Heating Featur...",[Water View: Pond]
